NLP

## Problem 1
Write a python program to perform the following tasks:

1) Estimate a bigram language model on “hw2_training_sets.txt”. Use your language model to compute:

    A. the probability,
    B. the probability normalized by sentence length,
    C. and the perplexity of the 5 sentences given in “test_set.txt”. 

The training and test files have already been tokenized.

2) Explain how you have handled unknown words that appear in the test set.

Submission:
- Submit your python code as “lastname-firstname-part1.py”.
- Submit an output file “output.txt” with each line containing all required values for the respective sentence, separated by comma. Note: This time, you are not allowed to use NLTK.

In [1]:
# Read in training set
lines = []
for line in open('training_set.txt'):
    if line.strip():
        lines.append(line.strip())
print('Read in lines:\n' , lines[0:5])

dict1 = {'<s>': 0, '</s>': 0, '<UNK>': 1} #Make sure '<UNK>' is part of dictionary for later
#punctuation = '\“\”\"\‘\'\''

listWords = [] #This will be the whole document word by word

for line in lines:
    dict1['<s>'] += 1
    listWords.append('<s>') #Append <s> and </s> to every line
    dict1['</s>'] += 1
    for word in line.split():       
        listWords.append(word)
        
        if word not in dict1:
            dict1[word] = 1
        else:
            dict1[word] += 1
    listWords.append('</s>')
    
#
#
#
#Substitute low-frequency words with '<UNK>' and keep count of them
'''
keysToDelete = []
for key in dict1:
    if dict1[key] == 1:
        keysToDelete.append(dict1[key])
        wordIndex = listWords.index(key)
        listWords[wordIndex] = '<UNK>'
        dict1['<UNK>'] += 1

for word in keysToDelete:
    del dict1['key']
'''  

'''
#Should I delete punctuation?
for char in punctuation:
    if char in dict1:
        del dict1[char]
for char in punctuation:
    if char in listWords:
        listWords.remove(char)
'''
#
# NO SMOOTHING FOR UNIGRAMS!!! ONLY BIGRAMS 
#

vocabulary = set(dict1)
#print('Vocabulary size:', len(vocabulary))

countTokens = sum(dict1.values())
print('\nWords in training_set:', len(listWords),'+ vocabulary size:', len(vocabulary),'=', countTokens, 'tokens in dict1')

print('Number of <s>:', dict1['<s>'])
print('Number of </s>:', dict1['</s>'])

#print('Count words in whole document:', len(listWords))
print('\nlistWords:\n', listWords[0:55])

#print('<s>' in vocabulary)
#print('\‘' in vocabulary)
#print('<UNK>' in listWords)
#print('' in dict1)
#print('' in vocabulary)
#print('' in listWords)

Read in lines:
 ['This is the genealogy of Jesus the Messiah the son of David , the son of Abraham :', 'Thus there were fourteen generations in all from Abraham to David , fourteen from David to the exile to Babylon , and fourteen from the exile to the Messiah .', 'This is how the birth of Jesus the Messiah came about : his mother Mary was pledged to be married to Joseph , but before they came together , she was found to be pregnant through the Holy Spirit .', 'Because Joseph her husband was faithful to the law , and yet did not want to expose her to public disgrace , he had in mind to divorce her quietly .', 'But after he had considered this , an angel of the Lord appeared to him in a dream and said , ‘ Joseph son of David , do not be afraid to take Mary home as your wife , because what is conceived in her is from the Holy Spirit .']

Words in training_set: 215745 + vocabulary size: 7146 = 215746 tokens in dict1
Number of <s>: 7728
Number of </s>: 7728

listWords:
 ['<s>', 'This', 'is

### Bigram Dict

A bigram language model requires bigram probabilities, which are calculated by taking the counts of word co-occurences and dividing them by the count of the current word. These values are then presented in a word-to-word matrix.

P(Wn, Wn-1) / P(Wn)

To get the bigram counts, I have to keep track of the previous word and the current word.

I could use an array, but I think a dictionary would make it easier
'''
rows = []
for i in range(len(vocabulary)):
    column = [0] * len(vocabulary)
    rows.append(column)
    
#print(rows[0][0:5])
'''

In [2]:
# Initialize word-to-word co-occurence matrix using vocabulary list
# USE DICT OF DICTS TO CREATE BIGRAM MATRIX

listVocab = list(vocabulary) #convert set to list
print('Vocab:', listVocab[0:10])

# Create dict with all bigrams from the training_set and their counts + one key for unknown bigrams in the test set
bigramDict = {'<UNK> <UNK>':0}
for word1, word2 in zip(listWords,listWords[1:]): #the whole document with the words in order
    bigram1 = word1 + ' ' + word2
    if bigram1 not in bigramDict:
        bigramDict[bigram1] = 1
    else:
        bigramDict[bigram1] += 1

# Add +1 Smoothing
for key in bigramDict:
    bigramDict[key] += 1

print('Size bigramDict:', len(bigramDict)) #56916
#print(bigramDict)
#print('<s> is' in bigramDict)
#print('<s> Love' in bigramDict)
#print('<s> love' in bigramDict)

Vocab: ['heart’s', 'Continue', 'shield', 'grabbed', '“', 'restless', 'wrongdoers', 'driving', 'acceptably', 'plague']
Size bigramDict: 56916


In [3]:
# To get the bigram probability, I have to take the bigram counts and divide them by the unigram counts
# P(word2 | word1) = bigramDict['word1 word2'] / unigramDict['word1'] + len(vocabulary)

bigramProbs = {}
for key in bigramDict:
    word1, word2 = key.split()
    probability = bigramDict[key] / (dict1[word1] + len(vocabulary))
    bigramProbs[key] = probability
        
print('Size bigramProbs:', len(bigramProbs))
#print(bigramProbs)

Size bigramProbs: 56916


In [14]:
# A. The probability of each sentence in the test set
# P(a,b,c,d) = P(a)•P(b|a)•P(c|ab)•P(d|abc)
# with Markov assumption aka, independence: P(a,b,c,d) = P(a)•P(b|a)•P(c|b)•P(d|c)
# i don't need P(a) because it's always 1

#Read in test set
linesT = [line.strip() for line in open('test_set.txt')]
#print(linesT, '\n')

vocabularySize = len(dict1)

pTestSet = []
sentenceLengths = []

for line in linesT:
    print('Line',linesT.index(line) + 1,'\n', line, '\n')
    words = line.split()
    words.insert(0, '<s>') #Add <s> and </s> to each line
    words.append('</s>')
    
    sentenceLengths.append(len(words))
    
    testBigrams = []
    for word1, word2 in zip(words,words[1:]): #the whole test document with the words in order
        bigram1 = word1 + ' ' + word2
        testBigrams.append(bigram1)
    print('Bigrams:\n', testBigrams, '\n')
    product = 1
    for i in range(len(testBigrams)):
        # I don't need this because all of my sentences start with '<s>'
        '''
        if i == 0: # just P(w1), this is '<s>' for every sentence, but this allows my program to work in any case
            if words[0] not in dict1: # first word is unknown 
                firstWord = '<UNK>'
            else:
                firstWord = words[0]
        '''
            
        firstWordProbability = 1
            #print('P(',firstWord,')=', round(firstWordProbability, 4))
        
        # for the rest of the words
        currentBigram = testBigrams[i]
                  
        if currentBigram not in bigramProbs:
            currentBigram = '<UNK> <UNK>'
        
        #print('Bigram:', currentBigram, 'Prob:', bigramProbs[currentBigram] )
        # Incorporate probability into product  
        product = product * bigramProbs[currentBigram] 
        
    print('Product:', product, '\n')
    pTestSet.append(product)

print('Probabilties test_set.txt:', pTestSet)
print('Sentence lengths:', sentenceLengths)

Line 1 
 When trouble or persecution comes because of the word , they quickly fall away . 

Bigrams:
 ['<s> When', 'When trouble', 'trouble or', 'or persecution', 'persecution comes', 'comes because', 'because of', 'of the', 'the word', 'word ,', ', they', 'they quickly', 'quickly fall', 'fall away', 'away .', '. </s>'] 

Product: 3.88993088935546e-41 

Line 2 
 The people were amazed when they saw the mute speaking , the crippled made well , the lame walking and the blind seeing . 

Bigrams:
 ['<s> The', 'The people', 'people were', 'were amazed', 'amazed when', 'when they', 'they saw', 'saw the', 'the mute', 'mute speaking', 'speaking ,', ', the', 'the crippled', 'crippled made', 'made well', 'well ,', ', the', 'the lame', 'lame walking', 'walking and', 'and the', 'the blind', 'blind seeing', 'seeing .', '. </s>'] 

Product: 2.5303743634734513e-69 

Line 3 
 When Pilate saw that he was getting nowhere , but that instead an uproar was starting , he took water and washed his hands in f

In [15]:
# B. The probability normalized by sentence length.
# Divide the probability of the whole sentence by the number of tokens in the sentence.
# Take the log of that number
# Take the exp of that number

import math 

#(math.log(14))

normalizedBySentenceLength = []

for i in range(len(pTestSet)):
    normalizedProb1 = math.exp(math.log(pTestSet[i]) / sentenceLengths[i])
    normalizedBySentenceLength.append(normalizedProb1)
    
print('P normalized by sentence length:', normalizedBySentenceLength)

P normalized by sentence length: [0.004196988294634119, 0.0022996458186199404, 0.0020519444566270088, 0.0025012798276624324, 0.0016460788978307649]


In [16]:
# C. The perplexity of the 5 sentences given in “test_set.txt”
# To calculate Perplexity, calculate the probability of the sentences. If the probability is high, the perplexity is low.

# Perplexity = the nth root of (1 / p(w1,w2,...wn) )
# multiply all the probabilities of the words from the sentence together.
# divide that number by one.
# take the nth root of that number. (n is the number of words in the sentence)

# import math

PP = []

for i in range(len(pTestSet)):
    perplexity = (1 / pTestSet[i]) ** (1/sentenceLengths[i])
    PP.append(perplexity)
    
print('Perplexity:', PP)

# Create output file
outputFile = open('output.txt','w') 

for i in range(len(pTestSet)):
    # 1)probability of sentence , 2) probability of sentence normalized by sentence length , 3) perplexity
    string1 = str(pTestSet[i]) + ',' + str(normalizedBySentenceLength[i]) + ',' + str(PP[i]) + '\n'
    outputFile.write(string1)
 
outputFile.close() 

Perplexity: [238.26609220676352, 434.8495720093622, 487.3426260493438, 399.795332349739, 607.5042947928068]


### 2) Explain how you have handled unknown words that appeared in the test set.

When I initialized my unigram and bigram dictionaries, I added a key called '<UNK>' in the unigram dictionary and '<UNK> <UNK>' in the bigram dictionary. When I come across a word in the test set that's not in the bigram dictionary, I use the bigram['<UNK> <UNK>'] / unigram['<UNK>'] probability.

## Problem 2
Sample a sentence from your language model and compute its perplexity.

In [10]:
# To sample a sentence, I start with '<s>' and set what possible words can come next. I stop when the chosen word is </s>.
# I use numpy.random.choice() to choose a possible next word.

import numpy as np

mySentence = '<s>'
currentWord = '<s>' #start at the beginning of the sentence

while currentWord != '</s>' and currentWord != '.' and currentWord != '?': 
#while currentWord != '</s>': 
    possibleNext = []
    for key in bigramDict:
        w1, w2 = key.split()
        if w1 == currentWord:
            possibleNext.append(w2)
    #print(len(possibleNext))

    array1 = np.array(possibleNext) #make an array of all the possibilities
    nextWord = np.random.choice(a = array1) #randomly select one of the possible words
    #print(nextWord)
    mySentence = mySentence + ' ' + nextWord
    currentWord = nextWord
    
print(mySentence)

<s> regarding the street , temperate , pay , gnashes his commands will greatly in paradise .


In [17]:
# Compute the perplexity of the sampled sentence.
# Perplexity = the nth root of (1 / p(w1,w2,...wn) )
# multiply all the probabilities of the words from the sentence together.
# divide that number by one.
# take the nth root of that number. (n is the number of words in the sentence)

allWords = mySentence.split()
sentenceBigrams = []
for word1, word2 in zip(allWords,allWords[1:]):
    bigram1 = word1 + ' ' + word2
    sentenceBigrams.append(bigram1)
#print(sentenceBigrams)

probabilityWords = 1
        
for i in range(len(sentenceBigrams)):
    currentBigram = sentenceBigrams[i]
    w1, w2 = currentBigram.split()
    
    if currentBigram not in bigramDict:
        currentBigram = '<UNK> <UNK>'

    probabilityWords = probabilityWords * bigramProbs[currentBigram]
    
perplexity1 = (1 / probabilityWords) ** (1/len(allWords))

print('Probability:', probabilityWords)
print('Perplexity:', perplexity1)

Probability: 1.4108363150719034e-58
Perplexity: 2529.134716046838
